In [1]:
import math
from collections import deque

# ---------------------- Level Printer ----------------------

def print_tree_levels(tree, title="TREE"):
    print(f"\n===== {title} =====\n")
    queue = deque([(tree, 0)])
    levels = {}

    while queue:
        node, lvl = queue.popleft()
        levels.setdefault(lvl, [])
        if isinstance(node, int):
            levels[lvl].append(str(node))
        else:
            label = "MAX" if lvl % 2 == 0 else "MIN"
            levels[lvl].append(label)
            for child in node:
                queue.append((child, lvl+1))

    for lvl, items in levels.items():
        print(f"Level {lvl}:  " + "   ".join(items))
    print()

# ---------------------- Alpha-Beta Pruning ----------------------

def alpha_beta(node, depth, alpha, beta, maximizingPlayer, pruned_nodes, level=0):
    if isinstance(node, int):
        return node

    if maximizingPlayer:
        value = -math.inf
        for child in node:
            result = alpha_beta(child, depth-1, alpha, beta, False, pruned_nodes, level+1)
            value = max(value, result)
            alpha = max(alpha, value)
            if beta <= alpha:
                # mark remaining siblings as pruned
                idx = node.index(child)
                for x in node[idx+1:]:
                    pruned_nodes.add(id(x))
                break
        return value
    else:
        value = math.inf
        for child in node:
            result = alpha_beta(child, depth-1, alpha, beta, True, pruned_nodes, level+1)
            value = min(value, result)
            beta = min(beta, value)
            if beta <= alpha:
                idx = node.index(child)
                for x in node[idx+1:]:
                    pruned_nodes.add(id(x))
                break
        return value

# ---------------------- Tree Reduction ----------------------

def prune_tree(node, pruned_nodes):
    if isinstance(node, int):
        return node
    new_children = []
    for child in node:
        if id(child) not in pruned_nodes:
            new_children.append(prune_tree(child, pruned_nodes))
    return new_children

# ---------------------- Tree from screenshot ----------------------

tree = [
    [ [10, 9], [14, 18] ],
    [ [5, 4], [50, 3] ]
]


# ---------------------- RUN ----------------------

print_tree_levels(tree, "ORIGINAL TREE (BEFORE PRUNING)")

pruned_nodes = set()
result = alpha_beta(tree, depth=3, alpha=-math.inf, beta=math.inf, maximizingPlayer=True, pruned_nodes=pruned_nodes)

pruned_tree = prune_tree(tree, pruned_nodes)

print_tree_levels(pruned_tree, "TREE AFTER PRUNING")
print("Final Optimal Value =", result)



===== ORIGINAL TREE (BEFORE PRUNING) =====

Level 0:  MAX
Level 1:  MIN   MIN
Level 2:  MAX   MAX   MAX   MAX
Level 3:  10   9   14   18   5   4   50   3


===== TREE AFTER PRUNING =====

Level 0:  MAX
Level 1:  MIN   MIN
Level 2:  MAX   MAX   MAX
Level 3:  10   9   14   5   4

Final Optimal Value = 10
